# Credit Risk 貸款風險
- Data source: https://www.openml.org/d/31
- This dataset classifies people described by a set of attributes as good or bad credit risks in Deutsche Mark.

In [2]:
import numpy as np
import pandas as pd

DATASET = pd.read_csv('dataset/dataset_31_credit-g.csv')
COL_NAME = [i for i in DATASET.columns ]

def convert_obj2cate(dataset, column):
    for i, typ in enumerate(dataset.dtypes):
        col = column[i]
        if typ.name == 'object':
            dataset[col] = dataset[col].astype('category')

convert_obj2cate(DATASET, COL_NAME)

DATASET = DATASET.rename(columns={"class": "risk"})
COL_NAME[-1] = "risk"

DATASET.dtypes

checking_status           category
duration                     int64
credit_history            category
purpose                   category
credit_amount                int64
savings_status            category
employment                category
installment_commitment       int64
personal_status           category
other_parties             category
residence_since              int64
property_magnitude        category
age                          int64
other_payment_plans       category
housing                   category
existing_credits             int64
job                       category
num_dependents               int64
own_telephone             category
foreign_worker            category
risk                      category
dtype: object

## Discrete Column

In [3]:
def get_cate_set(dataset, column):
    
    result = {}
    
    def get_set(col):
        _set = set([])
        for item in col:
            if item not in _set: _set.add(item)
        return _set

    for i, typ in enumerate(dataset.dtypes):
        col = column[i]
        if typ.name == 'category': 
            _set = get_set(dataset[col])
            result[col] = _set

    return result

def show_cate_attr_detail(cate_attr_detail):
    for attr, detail in cate_attr_detail.items():
        detail = list(detail)
        detail.sort()
        num = len(detail)
        content = ', '.join(detail).replace("\'", "")
        print('%20s:[%d]    %s' % (attr, num, content))

CATE_ATTR_DETAIL = get_cate_set(DATASET, COL_NAME)
show_cate_attr_detail(CATE_ATTR_DETAIL)


     checking_status:[4]    0<=X<200, <0, >=200, no checking
      credit_history:[5]    all paid, critical/other existing credit, delayed previously, existing paid, no credits/all paid
             purpose:[10]    domestic appliance, new car, used car, business, education, furniture/equipment, other, radio/tv, repairs, retraining
      savings_status:[5]    100<=X<500, 500<=X<1000, <100, >=1000, no known savings
          employment:[5]    1<=X<4, 4<=X<7, <1, >=7, unemployed
     personal_status:[4]    female div/dep/mar, male div/sep, male mar/wid, male single
       other_parties:[3]    co applicant, guarantor, none
  property_magnitude:[4]    life insurance, no known property, real estate, car
 other_payment_plans:[3]    bank, none, stores
             housing:[3]    for free, own, rent
                 job:[4]    high qualif/self emp/mgmt, unemp/unskilled non res, unskilled resident, skilled
       own_telephone:[2]    none, yes
      foreign_worker:[2]    no, yes
                

## Continuous Column

In [4]:
DATASET.describe()

,duration,credit_amount,installment_commitment,residence_since,age,existing_credits,num_dependents
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,20.903000,3271.258000,2.973000,2.845000,35.546000,1.407000,1.155000
std,12.058814,2822.736876,1.118715,1.103718,11.375469,0.577654,0.362086
min,4.000000,250.000000,1.000000,1.000000,19.000000,1.000000,1.000000
25%,12.000000,1365.500000,2.000000,2.000000,27.000000,1.000000,1.000000
50%,18.000000,2319.500000,3.000000,3.000000,33.000000,1.000000,1.000000
75%,24.000000,3972.250000,4.000000,4.000000,42.000000,2.000000,1.000000
max,72.000000,18424.000000,4.000000,4.000000,75.000000,4.000000,2.000000


## Column Summary

In [5]:
def col_summary(col_name, cate_detail):
    details = []
    for col in col_name:
        clean_col = col.replace("\'", "")
        detail = ""
        if col in cate_detail:
            d = cate_detail[col]
            detail = ', '.join(list(d)).replace("\'", "")
            details.append([clean_col, len(d), detail])
        else:
            detail = "int"
            details.append([clean_col, 1, detail])

    pd.set_option('max_colwidth',200)
    summary = pd.DataFrame(details, columns=['col','len','detail'])
    return summary

In [6]:
col_summary(COL_NAME, CATE_ATTR_DETAIL)

,col,len,detail
0,checking_status,4,">=200, 0<=X<200, <0, no checking"
1,duration,1,int
2,credit_history,5,"all paid, existing paid, delayed previously, critical/other existing credit, no credits/all paid"
3,purpose,10,"used car, repairs, domestic appliance, business, education, other, new car, radio/tv, furniture/equipment, retraining"
4,credit_amount,1,int
5,savings_status,5,"100<=X<500, no known savings, 500<=X<1000, >=1000, <100"
6,employment,5,"unemployed, 1<=X<4, 4<=X<7, >=7, <1"
7,installment_commitment,1,int
8,personal_status,4,"male mar/wid, male single, female div/dep/mar, male div/sep"
9,other_parties,3,"none, guarantor, co applicant"


## 類別比例

In [7]:
print("good:",len(DATASET.loc[DATASET['risk']=='good']))
print("bad:",len(DATASET.loc[DATASET['risk']=='bad']))

good: 700
bad: 300


## Column Summary

- 目標：確認是否有異常值或空值及離群值
ps. 可至 https://www.openml.org/d/31 看離散型資料是否異常

### 1 checking_status (O)
- 說明: 現有支票帳戶的狀態
- 欄位:(4) <0, 0<=X<200, >=200, no checking

### 2 duration (O)
- 說明: 持續時間（以月計）
- 欄位: int64

### 3 credit_history (O) 需專業討論
- 說明: 信用記錄（已貸記，已付清，延遲，重要帳戶)
- 欄位:(5) all paid, critical/other existing credit, delayed previously, existing paid, no credits/all paid

### 4 purpose (O)
- 說明: 信用目的（汽車，電視等）
- 欄位:(10) domestic appliance[家電], new car, used car, business, education, furniture/equipment, other, radio/tv, repairs, retraining

### 5 credit_amount (O)
- 說明: 貸方金額是指貸方承諾提供的最高金額，該金額是在本協議封面上的該條款之後規定的
- 欄位: int64

### 6 savings_status (O)
- 說明: 儲蓄賬戶/債券的狀態
- 欄位:(5) <100, 100<=X<500, 500<=X<1000, >=1000, no known savings

### 7 employment (O)
- 說明: 就業年數
- 欄位:(5) <1, 1<=X<4, 4<=X<7, >=7, unemployed

### 8 installment_commitment (O) 
- 說明: 分期付款率佔可支配收入的百分比
- 欄位: int64

### 9 personal_status (O)
- 說明: 個人身份（已婚，單身，...）和性別   div/sep:離婚
- 欄位:(4) female div/dep/mar, male div/sep, male mar/wid, male single

### 10 other_parties (O)
- 說明: 其他債務人/擔保人
- 欄位:(3) co applicant[共同申請人], guarantor[擔保人], none

### 11 residence_since (O)
- 說明: 自X年至今的居住地
- 欄位: int64

### 12 property_magnitude (O)
- 說明: 資產 eg 房地產
- 欄位:(4) life insurance, no known property, real estate, car

### 13 age (O)
- 說明: 年齡
- 欄位: int64

### 14 other_payment_plans (O)
- 說明: 其他分期付款計劃（銀行，商店）
- 欄位:(3) bank, none, stores

### 15 housing (O)
- 說明: 房屋（出租，擁有，...）
- 欄位:(3) for free, own, rent

### 16 existing_credits (O)
- 說明: 該銀行現有的信用分
- 欄位: int64

### 17 job (O)
- 說明: 工作狀況類型
- 欄位:(4) high qualif/self emp/mgmt, unemp/unskilled non res, unskilled resident, skilled

### 18 num_dependents (O)
- 說明: 負責人人數
- 欄位: int64

### 19 own_telephone (O)
- 說明: 有無電話
- 欄位:(2) none, yes

### 20 foreign_worker (O)
- 說明: 是否為外國工作者
- 欄位:(2) no, yes

### risk (O)
- 說明: 信用風險，為預測目標
- 欄位:(2) bad, good

## Feature Enginnering

- 離散型資料
    - 資料間無順序性的：one hot encoding <total column num: 40>

        - credit_history (5)
        - purpose (10)
        - personal_status (4)
        - other_parties (3)
        - property_magnitude (4)
        - other_payment_plans (3)
        - housing (3)
        - job (4)
        - own_telephone (2)
        - foreign_worker (2)
        
    - 資料間具順序性：label encoding <total column num: 3>
        - checking_status: <0 (-1), no checking (1), 0<=X<200 (2), >=200 (3)
        - savings_status: no known savings (0), <100(1), 100<=X<500(2), 500<=X<1000(3), >=1000(4)
        - employment: unemployed(0), <1(1), 1<=X<4(2), 4<=X<7(3), >=7(4)

- 連續型資料
    - Standardization 標準化 <total column num: 7>
        - duration
        - credit_amount
        - installment_commitment
        - residence_since
        - age
        - existing_credits
        - num_dependents

- Total columns: 50

## One hot encoding

In [8]:
def one_hot_feature_concat(cols, source_df, output_df):
    
    for col in cols:
        one_hot_df = pd.get_dummies(source_df[col])
        renames = {}
        for name in one_hot_df.columns:
            renames[name] = col +":"+ name.replace("\'", "")
        one_hot_df = one_hot_df.rename(columns=renames)
        one_hot_df = one_hot_df.astype('float32')
        output_df = output_df.join(one_hot_df)
    return output_df

LABEL = 'risk'
label_data = [1 if i == "good" else 0 for i in DATASET[LABEL]]
feature_enginnering_df = pd.DataFrame(label_data, columns=[LABEL])

one_hot_col = ['credit_history', 'purpose', 'personal_status', 'other_parties', \
               'property_magnitude', 'other_payment_plans', 'housing', 'job', 'own_telephone', 'foreign_worker']

feature_enginnering_df = one_hot_feature_concat(one_hot_col, DATASET, feature_enginnering_df)

In [9]:
feature_enginnering_df

,risk,credit_history:all paid,credit_history:critical/other existing credit,credit_history:delayed previously,credit_history:existing paid,credit_history:no credits/all paid,purpose:domestic appliance,purpose:new car,purpose:used car,purpose:business,...,housing:own,housing:rent,job:high qualif/self emp/mgmt,job:unemp/unskilled non res,job:unskilled resident,job:skilled,own_telephone:none,own_telephone:yes,foreign_worker:no,foreign_worker:yes
0,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0
1,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0
2,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
3,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0
4,0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
996,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
997,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0
998,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0


## Label Encoding

In [10]:
label_col = ['checking_status', 'savings_status', 'employment']

_dicts = [{"'<0'":-1.,"'no checking'":1.,"'0<=X<200'":2.,"'>=200'":3.},
          {"'no known savings'":0., "'<100'":1., "'100<=X<500'":2., "'500<=X<1000'":3., "'>=1000'":4. },
          {"unemployed":0., "'<1'":1., "'1<=X<4'":2., "'4<=X<7'":3., "'>=7'":4.}]

def label_feature_concat(label_cols, input_df, _dicts, output_df):
    data = None
    for idx, col in enumerate(label_cols):
        data = [ _dicts[idx][item] for item in input_df[col]]
        label_pd = pd.DataFrame(data, columns=[col])
        output_df = output_df.join(label_pd)
    return output_df

feature_enginnering_df = label_feature_concat(label_col, DATASET, _dicts , feature_enginnering_df)

In [11]:
feature_enginnering_df

,risk,credit_history:all paid,credit_history:critical/other existing credit,credit_history:delayed previously,credit_history:existing paid,credit_history:no credits/all paid,purpose:domestic appliance,purpose:new car,purpose:used car,purpose:business,...,job:unemp/unskilled non res,job:unskilled resident,job:skilled,own_telephone:none,own_telephone:yes,foreign_worker:no,foreign_worker:yes,checking_status,savings_status,employment
0,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,-1.0,0.0,4.0
1,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,1.0,2.0,1.0,2.0
2,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,3.0
3,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,1.0,-1.0,1.0,3.0
4,0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,1.0,-1.0,1.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,3.0
996,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,-1.0,1.0,2.0
997,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,4.0
998,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,-1.0,1.0,2.0


## Standardization Continuious

In [12]:
def standardization_feature_conat(cols, source_df, output_df):
    for col in cols:
        s = source_df[col]
        _std = s.std()
        _mean = s.mean()
        data = []
        for i in s:
            d = (i-_mean)/_std
            data.append(d)
        df = pd.DataFrame(data, columns=[col])
        output_df = output_df.join(df)
    return output_df

std_col = ['duration', 'credit_amount', 'installment_commitment',\
           'residence_since', 'age', 'existing_credits', 'num_dependents']

feature_enginnering_df = standardization_feature_conat(std_col, DATASET, feature_enginnering_df)

In [13]:
feature_enginnering_df

,risk,credit_history:all paid,credit_history:critical/other existing credit,credit_history:delayed previously,credit_history:existing paid,credit_history:no credits/all paid,purpose:domestic appliance,purpose:new car,purpose:used car,purpose:business,...,checking_status,savings_status,employment,duration,credit_amount,installment_commitment,residence_since,age,existing_credits,num_dependents
0,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-1.0,0.0,4.0,-1.235859,-0.744759,0.918018,1.046463,2.765073,1.026565,-0.428075
1,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,2.0,1.0,2.0,2.247070,0.949342,-0.869748,-0.765594,-1.190808,-0.704573,-0.428075
2,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,3.0,-0.738298,-0.416354,-0.869748,0.140434,1.182721,-0.704573,2.333701
3,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,-1.0,1.0,3.0,1.749509,1.633430,-0.869748,1.046463,0.831087,-0.704573,2.333701
4,0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,-1.0,1.0,2.0,0.256825,0.566380,0.024135,1.046463,1.534354,1.026565,2.333701
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,3.0,-0.738298,-0.543890,0.024135,1.046463,-0.399632,-0.704573,-0.428075
996,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,-1.0,1.0,2.0,0.754386,0.207509,0.918018,1.046463,0.391544,-0.704573,-0.428075
997,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,4.0,-0.738298,-0.874066,0.918018,1.046463,0.215727,-0.704573,-0.428075
998,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,-1.0,1.0,2.0,1.998289,-0.505275,0.918018,1.046463,-1.102900,-0.704573,-0.428075


## Shuffle then Split to train & validation
- 先打亂
- 切 train 與 valid 要做 10 fold cross validation，此資料無 test
    - train: 80% 訓練用
    - valid: 20% 調整參數用

In [15]:
from sklearn.utils import shuffle

feature_enginnering_df = shuffle(feature_enginnering_df, random_state=4)
# index=0 : not keeping index
feature_enginnering_df.to_csv('dataset/train.csv', index=0)

In [16]:
feature_enginnering_df

,risk,credit_history:all paid,credit_history:critical/other existing credit,credit_history:delayed previously,credit_history:existing paid,credit_history:no credits/all paid,purpose:domestic appliance,purpose:new car,purpose:used car,purpose:business,...,checking_status,savings_status,employment,duration,credit_amount,installment_commitment,residence_since,age,existing_credits,num_dependents
415,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,1.0,4.0,1.251947,0.910727,-0.869748,-0.765594,-0.047998,1.026565,2.333701
344,1,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,3.0,1.0,1.0,-0.904152,0.240101,-1.763631,-1.671623,0.127819,-0.704573,2.333701
602,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,1.0,3.0,0.256825,-0.508109,0.918018,1.046463,-0.135906,-0.704573,-0.428075
435,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,2.0,0.0,2.0,-0.738298,-0.633165,-0.869748,-1.671623,-0.927083,-0.704573,-0.428075
4,0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,-1.0,1.0,2.0,0.256825,0.566380,0.024135,1.046463,1.534354,1.026565,2.333701
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
566,0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,-1.0,1.0,3.0,-0.738298,-0.672843,-0.869748,0.140434,0.039910,-0.704573,-0.428075
982,1,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,3.0,2.0,2.0,0.008044,-0.123376,-1.763631,-1.671623,-0.663357,-0.704573,-0.428075
216,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,-1.0,1.0,3.0,-0.240737,-0.059254,0.024135,-1.671623,-0.399632,-0.704573,-0.428075
406,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,3.0,0.256825,-0.380573,0.918018,1.046463,0.655270,1.026565,2.333701


In [41]:
# def train_valid(df, valid_rate):
#     df_size = len(df.index)
#     valid_size = int(df_size * valid_rate)
#     train = df[:valid_size]
#     valid = df[valid_size:]
#     return train, valid

# train, valid = train_valid(df, 0.2)